In [13]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
            x = requests.get(uri).text
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            uri = 'https://' + '/'.join(mod[:-1]) + '/raw/main/' + mod[-1] + '.py'
            x = requests.get(uri).text
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass
    

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()

def img2b64(x):
    if isinstance(x, bytes):
        return base64.b64encode(x).decode()
    elif isinstance(x, str):
        with open(x, 'rb') as fr:
            return base64.b64encode(fr.read()).decode()
    raise

###
### Common ###
###

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')


CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed


In [74]:
a = { 'a': {'c': [12,2]}, 'b': 20}

In [79]:
a = [10, (10, 20), (20, 30)]
a[1]

(10, 20)

In [69]:
a

{'b': 20}

In [59]:
int(np.clip(10, 11, 20))

11

In [60]:
x = 

In [66]:
min(max(0, 31), 30)

30

In [35]:
b = a[a[:, 1].argsort()]
b

array([[0, 1, 2],
       [1, 2, 3],
       [5, 4, 3]])

In [51]:
c = np.array([[0,1,2],[1,2,3], [5,4,3]])
c

array([[0, 1, 2],
       [1, 2, 3],
       [5, 4, 3]])

In [52]:
for x in reversed(c):
    print(x)

[5 4 3]
[1 2 3]
[0 1 2]


In [46]:
for x in b:
    print(x)

[0 1 2]
[1 2 3]
[5 4 3]


In [41]:
for x in reversed(a[a[:, 1].argsort()]):
    print(x)

[5 4 3]
[1 2 3]
[0 1 2]


In [43]:
for x in a[a[:, 1].argsort()]:
    print(x)

[0 1 2]
[1 2 3]
[5 4 3]


In [1]:
from ipywidgets import RadioButtons, Text, Select, Dropdown, jslink, interactive, interact
from ipywidgets.widgets.widget import Widget, register, widget_serialization
from ipywidgets.widgets.widget_core import CoreWidget
from IPython.display import  display
import ipywidgets as widgets
# from ipywidgets.trait_types import CByteMemoryView         


from traitlets import CBytes, Unicode, Tuple, Instance, TraitError

Dropdown.value.tag(sync=True)

selection = Dropdown(options=[
    ('1', 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet/20210720143047.mp4'),
    ('2', 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet/20210720143047.mp4'),
    ('3', 'https://raceai.s3.didiyunapi.com/data/media/videos/repnet/20210720143047.mp4')])

class BytesText(Text):
    bvalue = CBytes(help="Bytes value").tag(sync=True)       

text = BytesText(value='a')
# l = jslink((text, 'value'), (selection, 'value'))
def test(a):
    print(a)
    
def observe_test(wdg):
    print(wdg)
    wdg['owner'].bvalue = wdg['new'].encode('utf-8')
    
o = text.observe(observe_test, 'value')
    
# interact(test, a=text);

video = widgets.Video(format='url')

l = jslink((text, 'bvalue'), (video, 'value'))

# display(selection, video)
display(text, video)

BytesText(value='a')

Video(value=b'', format='url')